In [1]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

/home/roy/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [2]:
data = np.loadtxt('mach_test_binary_2_add.csv', delimiter=',')


In [27]:
samples=2000000
X = data[:samples,1:-2]


In [28]:
Y = data[:samples,-2:]
PC_Address=data[:samples,0]

In [32]:
print(PC_Address.shape,Y.shape)
x=int(PC_Address[1000]%10000)
print(x)
print(PC_Address[x])


(2000000,) (2000000, 2)
6662
7273386.0


In [33]:
n_nodes_hl1 = 100
n_nodes_hl2 = 100
n_nodes_hl3 = 100
n_classes = 2
batch_size = 100
GHR=np.zeros((1,100))
LHR=np.zeros((10000,16))
result=np.zeros((1,samples))


x = tf.placeholder('float', [None, 148])
y = tf.placeholder('float',[None,2])
def neural_network_model(data):
    hidden_1_layer = {'weights':tf.Variable(tf.random_normal([148, n_nodes_hl1])),
                      'biases':tf.Variable(tf.random_normal([n_nodes_hl1]))}

    hidden_2_layer = {'weights':tf.Variable(tf.random_normal([n_nodes_hl1, n_nodes_hl2])),
                      'biases':tf.Variable(tf.random_normal([n_nodes_hl2]))}

    hidden_3_layer = {'weights':tf.Variable(tf.random_normal([n_nodes_hl2, n_nodes_hl3])),
                      'biases':tf.Variable(tf.random_normal([n_nodes_hl3]))}

    output_layer = {'weights':tf.Variable(tf.random_normal([n_nodes_hl3, n_classes])),
                    'biases':tf.Variable(tf.random_normal([n_classes])),}
    l1 = tf.add(tf.matmul(data,hidden_1_layer['weights']), hidden_1_layer['biases'])
    l1 = tf.nn.relu(l1)

    l2 = tf.add(tf.matmul(l1,hidden_2_layer['weights']), hidden_2_layer['biases'])
    l2 = tf.nn.relu(l2)

    l3 = tf.add(tf.matmul(l2,hidden_3_layer['weights']), hidden_3_layer['biases'])
    l3 = tf.nn.relu(l3)

    output = tf.matmul(l3,output_layer['weights']) + output_layer['biases']

    return output


def train_neural_network(x):
    prediction = neural_network_model(x)
    #cost = tf.reduce_mean( tf.nn.l2_loss(prediction-y) )
    cost = tf.reduce_mean( tf.nn.softmax_cross_entropy_with_logits(logits=prediction, labels=y) )
    optimizer = tf.train.AdamOptimizer().minimize(cost)
    train_error= tf.equal(tf.argmax(prediction, 1), tf.argmax(y, 1))
    #hm_epochs = 10
    hm_epochs = 1
    temp=0
    
    with tf.Session() as sess:

        sess.run(tf.global_variables_initializer())
        for epoch in range(hm_epochs):
            epoch_loss = 0
            acc=0

            for i in range(int(samples)):
                 #local history table address index found using the address
                add=int(PC_Address[i]%10000)
                # the input vector to the Neural network that has GHR, LHR and PC
                epoch_x=np.concatenate((X[i:i+1],GHR,LHR[add:add+1]),axis=1)

                epoch_y=Y[i:i+1]
                epoch_y.shape=(1,2)

                _, c = sess.run([optimizer, cost], feed_dict={x: epoch_x, y: epoch_y})
                #This is for checking the result output
                eq=sess.run([train_error],feed_dict={x:epoch_x, y:epoch_y})
                acc+=eq[0]
                #i=i+1
                epoch_loss += c
                #updata GHR
                GHR[0,:-1]=GHR[0,1:]
                GHR[0,-1]=Y[i:i+1,0]
                result[0,i]=acc

                #update LHR
                LHR[add,:-1]=LHR[add,1:]
                LHR[add,-1]=Y[i:i+1,0]


train_neural_network(x)

Epoch 0 completed out of 1 loss: 1507967.1132
Accuracy:  [ 0.95852448]
Accuracy:  [1917048]
[[  0.00000000e+00   1.00000000e+00   1.00000000e+00 ...,   1.91704600e+06
    1.91704700e+06   1.91704800e+06]]


In [34]:
temp=0
print(len(result[0]))
for i in range(int(len(result[0])/10000)):
    print("cycle accuracy",(result[0,i*10000]-temp)/10000)
    temp=result[0,i*10000]

2000000
cycle accuracy 0.0
cycle accuracy 0.881
cycle accuracy 0.9705
cycle accuracy 0.9571
cycle accuracy 0.9806
cycle accuracy 0.9492
cycle accuracy 0.9664
cycle accuracy 0.9399
cycle accuracy 0.9079
cycle accuracy 0.9476
cycle accuracy 0.9681
cycle accuracy 0.9219
cycle accuracy 0.8511
cycle accuracy 0.9582
cycle accuracy 1.0
cycle accuracy 0.9736
cycle accuracy 0.9905
cycle accuracy 0.9838
cycle accuracy 0.9557
cycle accuracy 0.9571
cycle accuracy 0.9579
cycle accuracy 0.9348
cycle accuracy 0.9602
cycle accuracy 0.9684
cycle accuracy 0.9702
cycle accuracy 0.924
cycle accuracy 0.9448
cycle accuracy 0.9714
cycle accuracy 0.9602
cycle accuracy 0.9501
cycle accuracy 0.9558
cycle accuracy 0.9522
cycle accuracy 0.9619
cycle accuracy 0.9568
cycle accuracy 0.9664
cycle accuracy 0.9586
cycle accuracy 0.9606
cycle accuracy 0.9662
cycle accuracy 0.9558
cycle accuracy 0.9602
cycle accuracy 0.9596
cycle accuracy 0.9641
cycle accuracy 0.9658
cycle accuracy 0.9606
cycle accuracy 0.9614
cycle accu